In [1]:
import os
import random
import pickle
import torch
import torchvision
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

In [4]:
import torchvision.transforms as transforms
from torch import nn

In [5]:
class TripletNet(torch.nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n

In [7]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv0 = nn.Sequential(
            nn.Conv2d(1, 64, (128, 3), padding=(0, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.conv1 = nn.Sequential(
            nn.Conv2d(64, 64, (1, 3), padding=(0,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, (1, 3), padding=(0,1)),
            nn.BatchNorm2d(64)
        )

        self.skip1 = nn.Sequential(
            nn.Conv2d(64, 64, (1, 1)),
            nn.BatchNorm2d(64)
        )

        self.sum1 = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, (1, 3), stride=(1,2), padding=(0,1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, (1, 3), padding=(0,1)),
            nn.BatchNorm2d(128)
        )

        self.skip2 = nn.Sequential(
            nn.Conv2d(64, 128, (1, 1), stride=(1,2)),
            nn.BatchNorm2d(128)
        )

        self.sum2 = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, (1, 3), stride=(1,2), padding=(0,1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, (1, 3), padding=(0,1)),
            nn.BatchNorm2d(256)
        )

        self.skip3 = nn.Sequential(
            nn.Conv2d(128, 256, (1, 1), stride=(1,2)),
            nn.BatchNorm2d(256)
        )

        self.sum3 = nn.Sequential(
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, (1, 3), stride=(1,2), padding=(0,1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, (1, 3), padding=(0,1)),
            nn.BatchNorm2d(512)
        )

        self.skip4 = nn.Sequential(
            nn.Conv2d(256, 512, (1, 1), stride=(1,2)),
            nn.BatchNorm2d(512)
        )

        self.sum4 = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.global_pooling = nn.AvgPool2d((1, 156))

        self.dense = nn.Linear(512, 4096)


    def forward(self, x):
        x0 = self.conv0(x)
        x1 = self.sum1(self.conv1(x0) + self.skip1(x0))
        x2 = self.sum2(self.conv2(x1) + self.skip2(x1))
        x3 = self.sum3(self.conv3(x2) + self.skip3(x2))
        x4 = self.sum4(self.conv4(x3) + self.skip4(x3))
        x5 = self.global_pooling(x4)

        x6 = x5.view(x5.shape[0], -1)
        x7 = self.dense(x6)
        return x7

In [8]:
model = torch.load('../data/pickles/models/custom_resnet_27.12.0.16.pth')

In [9]:
embd_net = model.embeddingnet

In [10]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [11]:
# Normalize training set together with augmentation
one_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
#     transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.0069], [0.0033])
])

train_dir = '../data/spectrograms/train/train/'
val_dir = '../data/spectrograms/train/val/'
test_dir = '../data/spectrograms/test/'

batch_size = 16
test_dataset = ImageFolderWithPaths(test_dir, one_transform)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

train_dataset = ImageFolderWithPaths(train_dir, one_transform)
val_dataset = ImageFolderWithPaths(val_dir, one_transform)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=8
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

# class_names = train_dataset.classes


# Loading Tiny ImageNet dataset
# trplt_path = '../pickles_for_git/triplets.p'
# batch_szie = 32
# trainset = TripletDataset(triplet_path=trplt_path, transform=one_transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_szie, num_workers=8)

# testset = TripletImageLoader(
#     base_path=root, triplets_filename="", transform=transform_test, train=False)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=batch_size_test, num_workers=32)

In [12]:
len(test_dataloader), len(test_dataset)

(7, 97)

In [13]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs, x, path in test_loader:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = torch.nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [14]:
embd_net = embd_net.to(device)

In [15]:
with torch.no_grad():
    logits = []

    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        break

In [16]:
logits[0][0].shape

torch.Size([4096])

In [17]:
with torch.no_grad():
    logits = []
    path_list = list()
    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [18]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in val_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [19]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in train_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [20]:
path_list[0][0]

'../data/spectrograms/test/unknown/000190.png'

In [21]:
logits[0][0].shape[0]

4096

In [22]:
emb_list = list()
for i in logits:
    for tens in i:
        emb_list.append(tens.numpy())

In [23]:
embeddings = pd.DataFrame(emb_list)

In [24]:
track_list = list()
for i in path_list:
    for path in i:
        name = path.split('/')[-1]
        ids = name.split('.')[0]
        track_list.append(ids)

In [25]:
embeddings['id'] = track_list

In [26]:
embeddings = embeddings[['id'] + list(np.arange(logits[0][0].shape[0]))]

In [27]:
embeddings

,id,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,000190,0.239317,-0.397095,0.066670,-0.112857,0.267753,-0.052551,0.096670,-0.045432,0.172941,...,0.062273,-0.054813,-0.580411,-0.060844,0.283575,-0.322482,0.536448,-0.547134,0.435944,-0.009193
1,000194,0.228010,-0.267925,0.050254,-0.064885,0.204552,-0.129576,0.097313,0.054549,0.160086,...,0.123470,0.036018,-0.554675,-0.084867,0.241101,-0.407837,0.459143,-0.405547,0.325219,0.030739
2,000667,0.172511,-0.411962,-0.189085,-0.207302,0.240760,-0.356008,-0.125368,0.159223,0.145956,...,0.091740,0.235835,-0.380354,0.122873,0.403869,-0.645875,0.179191,-0.043992,-0.246042,0.162271
3,001040,0.082101,-0.374198,-0.069975,-0.125310,0.116177,-0.230725,-0.045782,0.068693,0.189867,...,0.023682,0.061822,-0.319704,-0.105478,0.274142,-0.537587,0.266508,-0.130056,0.051768,0.177883
4,001686,0.097401,-0.337461,-0.043460,-0.049395,0.358293,-0.022668,0.214166,-0.157234,0.158639,...,0.095537,0.080308,-0.397135,0.167263,0.297460,-0.427000,0.659261,-0.343497,0.185148,0.160289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7989,149416,0.030924,-0.247550,-0.178629,-0.061112,0.027577,-0.162658,-0.221465,-0.065798,0.129087,...,0.195687,0.089321,-0.291216,-0.142482,0.205989,-0.573224,0.384657,-0.033724,-0.058859,0.158304
7990,149417,0.036764,-0.458351,-0.088213,-0.177309,-0.051371,-0.145914,-0.140578,-0.086203,0.120065,...,0.140031,0.064523,-0.262126,-0.076916,0.275563,-0.584401,0.395788,-0.099448,0.052889,0.137360
7991,149452,0.104525,-0.469663,0.084858,-0.304183,0.086875,-0.330575,-0.015224,-0.085978,0.090347,...,0.128999,0.089443,-0.236963,-0.058777,0.350236,-0.518301,0.177216,-0.213805,-0.100781,0.229377
7992,149488,0.013139,-0.466050,-0.014817,-0.071389,-0.013317,-0.200836,0.081551,-0.161710,-0.112145,...,0.090962,0.241779,-0.107526,-0.191463,0.313202,-0.453634,0.384773,0.071279,-0.046346,0.256486


In [28]:
embeddings.to_csv('../data/csv/custom_resnet_triplet_embeddings.csv', index=False)